# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure function/wrapper
gmaps.configure(api_key=g_key) 

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Input data file
input_data_file = "output_data/cities.csv" 
cities_weather_df = pd.read_csv(input_data_file)
cities_weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Cape Town,20,ZA,1579250107,69,-33.93,18.42,84.99,29.97
1,Castro,100,BR,1579250276,96,-24.79,-50.01,60.42,6.96
2,Igarka,100,RU,1579250276,93,67.47,86.58,10.65,15.39
3,Santa Isabel,75,BR,1579250277,100,-23.32,-46.22,71.60,4.70
4,Lagoa,100,PT,1579250277,90,39.05,-27.98,56.95,10.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Define coordinates and weights for the heatmap
coordinates = cities_weather_df[['Lat', 'Lon']]
humid_weights = cities_weather_df['Humidity']

In [4]:
# Plot Heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humid_weights, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities based on maximum temperature whereas it is between 60 and 80
narrowed_city_df = cities_weather_df.loc[(cities_weather_df['Max Temp'] > 60) & (cities_weather_df['Max Temp'] < 80),:]
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
1,Castro,100,BR,1579250276,96,-24.79,-50.01,60.42,6.96
3,Santa Isabel,75,BR,1579250277,100,-23.32,-46.22,71.60,4.70
6,Vaini,90,TO,1579250277,94,-21.20,-175.20,77.00,11.41
10,Bredasdorp,100,ZA,1579250278,69,-34.53,20.04,75.20,17.22
15,Les Cayes,28,HT,1579250279,72,18.20,-73.75,79.54,13.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Copy narrowed city onto a hotel data frame
coordinates = narrowed_city_df[['Lat', 'Lon']]
hotel_df = narrowed_city_df.copy()
hotel_df.reset_index()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
1,Castro,100,BR,1579250276,96,-24.79,-50.01,60.42,6.96,
3,Santa Isabel,75,BR,1579250277,100,-23.32,-46.22,71.60,4.70,
6,Vaini,90,TO,1579250277,94,-21.20,-175.20,77.00,11.41,
10,Bredasdorp,100,ZA,1579250278,69,-34.53,20.04,75.20,17.22,
15,Les Cayes,28,HT,1579250279,72,18.20,-73.75,79.54,13.42,


In [7]:
# Build parameter for Google Places API
search_params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key
}

In [8]:
#Iterate through Hotel Data frame and request for the Hotel name one by one, if any results are returned

for index, rowvals in hotel_df.iterrows():
    gplace_base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    latitude = rowvals['Lat']
    longitude = rowvals['Lon']
    
    search_params['location'] = f"{latitude},{longitude}"
    response = requests.get(gplace_base_url, params=search_params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print(f"Fething for {rowvals['City']}... Hotel {response['results'][0]['name']} found!")
    except (KeyError, IndexError):
        print("No results returned... Skipping")

Fething for Castro... Hotel Hotel Buganville Express found!
Fething for Santa Isabel... Hotel Recanto La Rosita found!
Fething for Vaini... Hotel Paradise First Hotel found!
Fething for Bredasdorp... Hotel Bredasdorp Country Manor found!
Fething for Les Cayes... Hotel Hôtel Le Méridien found!
Fething for Kapaa... Hotel Sheraton Kauai Coconut Beach Resort found!
Fething for Kunming... Hotel Hotel Kunming found!
Fething for Sur... Hotel Sur Plaza Hotel found!
Fething for Rikitea... Hotel Pension Maro'i found!
Fething for Port Elizabeth... Hotel Millbury Guesthouse found!
Fething for Mataura... Hotel Ellie's Villa found!
Fething for Vila Franca do Campo... Hotel Pestana Bahia Praia found!
Fething for Biloela... Hotel Biloela Countryman Motel found!
Fething for Bandarbeyla... Hotel REER XOOSHLE GROUP found!
Fething for Puerto Madero... Hotel Mision Surf Mexico found!
Fething for Nelson Bay... Hotel Hotel Nelson - Formerly The Nelson Resort found!
Fething for Kwang Binh... Hotel Mường Thanh

Fething for Namwala... Hotel Corner Point Lodge found!
Fething for Champerico... Hotel Hotel y restaurante "El Diamante" found!
Fething for San Juan del Sur... Hotel Dreamsea Surf Resort Nicaragua found!
Fething for Puerto Colombia... Hotel Hotel Puerto Colombia found!
Fething for Kirando... Hotel Lake Shore Lodge, Tanganyika found!
Fething for Victor Harbor... Hotel The Anchorage Hotel found!
Fething for Iquique... Hotel Hotel Terrado Cavancha found!
Fething for Poso... Hotel Ancyra Hotel By Continent - Poso found!


In [9]:
# Remove null fields and See results
hotel_df = hotel_df.dropna()
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
1,Castro,100,BR,1579250276,96,-24.79,-50.01,60.42,6.96,Hotel Buganville Express
3,Santa Isabel,75,BR,1579250277,100,-23.32,-46.22,71.60,4.70,Recanto La Rosita
6,Vaini,90,TO,1579250277,94,-21.20,-175.20,77.00,11.41,Paradise First Hotel
10,Bredasdorp,100,ZA,1579250278,69,-34.53,20.04,75.20,17.22,Bredasdorp Country Manor
15,Les Cayes,28,HT,1579250279,72,18.20,-73.75,79.54,13.42,Hôtel Le Méridien
...,...,...,...,...,...,...,...,...,...,...
551,Puerto Colombia,12,CO,1579250376,83,10.99,-74.95,77.00,14.99,Hotel Puerto Colombia
554,Kirando,96,TZ,1579250377,70,-7.42,30.60,75.67,2.77,"Lake Shore Lodge, Tanganyika"
555,Victor Harbor,1,AU,1579250377,37,-35.57,138.62,69.80,17.22,The Anchorage Hotel
558,Iquique,95,CL,1579250377,64,-20.22,-70.14,69.80,2.64,Hotel Terrado Cavancha


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig = gmaps.figure()


# Plot Heatmap
temps = hotel_df['Max Temp']
heat_layer_hotels = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=5, point_radius=1)
fig.add_layer(heat_layer_hotels)
fig.add_layer(marker_layer)


# Display Map
fig

Figure(layout=FigureLayout(height='420px'))